DATA CLEANING

In [2]:
# ====== 0. IMPORT & CONFIG ======
!pip install textblob
import os
import re
import unicodedata
import numpy as np
import pandas as pd
from textblob import TextBlob
from collections import Counter

DATA_DIR = "C:\\Users\\ADMIN\\Documents\\Khóa luận\\MovieTweetings Dataset"      # thư mục chứa ratings.dat & movies.dat
MIN_USER_INTERACTIONS = 5  # user có < 5 rating: bỏ qua test
CV_FOLDS = 3               # số fold Temporal Cross-Validation
TRAIN_RATIOS = [0.7, 0.75, 0.8]  # tỉ lệ train/test cho từng fold

def sec(title): 
    print("\n" + "="*12 + f" {title.upper()} " + "="*12)
#Đây là hàm tiện ích để in ra tiêu đề mỗi bước trong console cho dễ nhìn.


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


- Fold 1: 70% train, 30% test
- Fold 2: 75% train, 25% test
- Fold 3: 80% train, 20% test 

🔹 Mục tiêu: kiểm tra xem mô hình ổn định thế nào khi lượng dữ liệu train thay đổi theo thời gian.

Để đánh giá tính ổn định của mô hình trong điều kiện thời gian thực, khóa luận sử dụng Temporal Cross-Validation với 3 tỉ lệ chia dữ liệu: 70/30, 75/25 và 80/20.

Ba tỉ lệ này lần lượt mô phỏng các giai đoạn khác nhau trong vòng đời của hệ thống gợi ý:

- Khi hệ thống mới khởi tạo (ít dữ liệu quá khứ – Fold 1).

- Khi hệ thống dần tích lũy thêm dữ liệu (Fold 2).

- Khi hệ thống đã vận hành ổn định với lượng dữ liệu lớn (Fold 3).

Cách chia này đảm bảo vừa tôn trọng tính thời gian của dữ liệu, vừa đánh giá được khả năng học và tổng quát hóa của mô hình khi lượng dữ liệu huấn luyện thay đổi.

MIN_USER_INTERACTIONS = 5

- Một số user chỉ có vài rating (1–2 phim), nếu chia 80/20 thì test rỗng (vì 20% của 2 dòng = 0.4).

- Cho nên: nếu user có ít hơn 5 rating, ta cho hết vào train, không chia test.

- Tránh lỗi khi mô hình học.

In [4]:
sec("LOAD DỮ LIỆU")

def read_dat(fname, cols):
    path = os.path.join(DATA_DIR, fname) #Ghép thư mục gốc (DATA_DIR) với tên file (fname) → tạo đường dẫn đầy đủ.
    if not os.path.exists(path): #Kiểm tra file có tồn tại không.
        raise FileNotFoundError(f"Không tìm thấy file: {path}")
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            return pd.read_csv(path, sep="::", engine="python", names=cols, encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, sep="::", engine="python", names=cols, encoding="latin-1", on_bad_lines="skip")
#Đọc file .dat bằng pandas.sep="::": vì file MovieTweetings không phải CSV thông thường (dấu phẩy ,) mà dùng :: làm dấu phân cách.

#Hàm read_dat() trả về một DataFrame sạch, có tên cột đúng, không lỗi mã hóa.
ratings = read_dat("ratings.dat", ["UserID","MovieID","Rating","Timestamp"])
movies  = read_dat("movies.dat",  ["MovieID","Title","Genres"])

print("ratings:", ratings.shape, "| movies:", movies.shape)

ratings["Datetime"] = pd.to_datetime(ratings["Timestamp"], unit="s", errors="coerce")
ratings = ratings.astype({"UserID":"int32","MovieID":"int32","Rating":"int16","Timestamp":"int64"})
movies["MovieID"] = movies["MovieID"].astype("int32")


============ LOAD DỮ LIỆU ============
ratings: (921398, 4) | movies: (38018, 3)


Bảng ratings: 921 398 bản ghi và 4 thuộc tính (UserID, MovieID, Rating, Timestamp). Mỗi bản ghi biểu diễn một lượt đánh giá mà người dùng dành cho một bộ phim cụ thể tại thời điểm xác định.

Bảng movies: 38 018 bản ghi và 3 thuộc tính (MovieID, Title, Genres). Mỗi dòng tương ứng với một bộ phim duy nhất kèm tiêu đề và danh sách thể loại.

In [5]:
sec("HIỂU DỮ LIỆU")

print("Số user:", ratings["UserID"].nunique(),
      "| Số movie:", ratings["MovieID"].nunique(),
      "| Tổng rating:", len(ratings))

print("Khoảng thời gian rating:", ratings["Datetime"].min(), "→", ratings["Datetime"].max())

print("\nPhân phối rating (0–10):")
print(ratings["Rating"].value_counts().sort_index())

print("\nMô tả thống kê rating:")
print(ratings[["Rating"]].describe())


============ HIỂU DỮ LIỆU ============
Số user: 71707 | Số movie: 38013 | Tổng rating: 921398
Khoảng thời gian rating: 2013-02-28 14:38:27 → 2021-11-24 05:41:29

Phân phối rating (0–10):
Rating
0        281
1      10814
2       9223
3      15487
4      28193
5      69747
6     120370
7     206680
8     222146
9     130106
10    108351
Name: count, dtype: int64

Mô tả thống kê rating:
              Rating
count  921398.000000
mean        7.312627
std         1.852559
min         0.000000
25%         6.000000
50%         7.000000
75%         9.000000
max        10.000000


Ý nghĩa:
- Cột “Rating” là điểm người dùng chấm cho phim, từ 0 đến 10.
- Bảng này đếm tần suất (số lượt) của từng điểm số.

→ Dữ liệu thiên lệch về phía cao (positive bias), tức là phần lớn người dùng đánh giá phim họ thích.

→ Khi huấn luyện mô hình, cần lưu ý điều này (có thể dùng normalization hoặc implicit feedback modeling).

Bộ dữ liệu MovieTweetings chứa 921 398 lượt đánh giá từ 71 707 người dùng cho 38 013 bộ phim, được thu thập trong giai đoạn 2013 – 2021.

Phân phối điểm đánh giá cho thấy phần lớn người dùng chấm điểm cao (trên 7), với điểm trung bình là 7.31 và độ lệch chuẩn 1.85.
Điều này phản ánh hiện tượng thiên lệch tích cực (positive bias) thường gặp trong các bộ dữ liệu gợi ý, khi người dùng có xu hướng chỉ đánh giá những phim họ yêu thích.

Dữ liệu này bao phủ giai đoạn dài và đủ lớn để thực hiện các thí nghiệm với mô hình Collaborative Filtering, Content-Based, và Hybrid, cũng như áp dụng chia dữ liệu theo thời gian (Temporal Cross-Validation).

In [8]:
sec("KIỂM TRA CHẤT LƯỢNG")

# Missing check
print("Missing ratings:\n", ratings.isna().sum())
print("Missing movies:\n", movies.isna().sum())

# Out-of-range rating
bad_rating = ~ratings["Rating"].between(0,10)
print("Số rating ngoài [0,10]:", int(bad_rating.sum()))

# Duplicates
dup_exact = ratings.duplicated(subset=["UserID","MovieID","Timestamp"]).sum()
dup_ui = ratings.duplicated(subset=["UserID","MovieID"]).sum()
print("Duplicate exact (User,Movie,Time):", int(dup_exact))
print("Duplicate (User,Movie):", int(dup_ui))

# MovieID integrity
unknown_movies = np.setdiff1d(ratings["MovieID"].unique(), movies["MovieID"].unique())
print("MovieID trong ratings không có ở movies:", len(unknown_movies))

# Check for mojibake
mojibake_patterns = [r"Ã.", r"Â", r"â€", r"â€“", r"â€”", r"â€™", r"â€œ", r"â€"]
mask_mojibake = movies["Title"].astype(str).str.contains("|".join(mojibake_patterns))
print("Titles nghi lỗi mã hóa (mojibake):", int(mask_mojibake.sum()))

# Non-ASCII ratio
def non_ascii_ratio(s): 
    s = str(s)
    return sum(ord(ch) > 127 for ch in s)/max(1,len(s))
movies["non_ascii_ratio"] = movies["Title"].apply(non_ascii_ratio)
print("Titles có ký tự non-ASCII (>0):", int((movies["non_ascii_ratio"]>0).sum()))

# Genres missing
mask_genres_missing = movies["Genres"].isna() | (movies["Genres"].astype(str).str.strip()=="")
print("Genres trống/missing:", int(mask_genres_missing.sum()))


============ KIỂM TRA CHẤT LƯỢNG ============
Missing ratings:
 UserID       0
MovieID      0
Rating       0
Timestamp    0
Datetime     0
dtype: int64
Missing movies:
 MovieID     0
Title       0
Genres     54
dtype: int64
Số rating ngoài [0,10]: 0
Duplicate exact (User,Movie,Time): 0
Duplicate (User,Movie): 0
MovieID trong ratings không có ở movies: 0
Titles nghi lỗi mã hóa (mojibake): 1
Titles có ký tự non-ASCII (>0): 2240
Genres trống/missing: 54


Bộ dữ liệu phim tương đối sạch, chỉ còn 54 dòng bị thiếu trường “Genres” (thể loại).
=> Điền giá trị mặc định "Unknown".

Phát hiện 1 tiêu đề phim có dấu hiệu bị lỗi mã hóa ký tự (ví dụ “PokÃ©mon” thay vì “Pokémon”).
=> Đây là hậu quả của sai encoding (UTF-8 vs Latin-1) trong dữ liệu gốc.

Có 2 240 tiêu đề chứa ký tự đặc biệt ngoài bảng chữ cái ASCII cơ bản (ví dụ: é, ñ, ü, ’,…).
=> Điều này hoàn toàn bình thường, vì nhiều phim có tên tiếng nước ngoài (Pháp, Nhật, Tây Ban Nha,…). Không cần loại bỏ — chỉ cần đảm bảo xử lý Unicode đúng chuẩn UTF-8 trong toàn pipeline.

In [9]:
sec("LÀM SẠCH DỮ LIỆU")

# 1. Sửa lỗi mã hóa thường gặp
def fix_mojibake(s):
    if not isinstance(s, str): return s
    replacements = {
        "PokÃ©mon":"Pokémon", "AmÃ©lie":"Amélie", "FranÃ§ais":"Français",
        "SeÃ±or":"Señor", "JosÃ©":"José", "BeyoncÃ©":"Beyoncé"
    }
    for bad, good in replacements.items(): s = s.replace(bad, good)
    return s

movies["Title"] = movies["Title"].apply(fix_mojibake)

# 2. Chuẩn hóa Unicode
movies["Title"] = movies["Title"].apply(lambda x: unicodedata.normalize("NFKC", x) if isinstance(x,str) else x)

# 3. Loại ký tự đặc biệt không cần thiết
def clean_text(s):
    if not isinstance(s,str): return s
    s = re.sub(r"[^0-9A-Za-zÀ-ỹ’\'\-\.\,\!\?\:\;\(\)\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s
movies["Title"] = movies["Title"].apply(clean_text)

# 4. Điền Genres thiếu
movies["Genres"] = movies["Genres"].fillna("Unknown").replace("", "Unknown")

# 5. Loại duplicate MovieID và (Title, Year)
def extract_year(title):
    if not isinstance(title,str): return np.nan
    m = re.search(r"\((\d{4})\)\s*$", title)
    return int(m.group(1)) if m else np.nan
movies["YearFromTitle"] = movies["Title"].apply(extract_year).astype("Int64")

movies = movies.sort_values(["MovieID","Genres"], ascending=[True,True])
movies = movies.drop_duplicates(subset=["MovieID"], keep="last")
movies = movies.drop_duplicates(subset=["Title","YearFromTitle"], keep="first")

# 6. Viết hoa chuẩn
movies["Title"] = movies["Title"].str.strip().apply(lambda s: s.title() if isinstance(s,str) else s)

# 7. Kết hợp genres list
movies["GenresList"] = movies["Genres"].apply(lambda x: [g.strip() for g in str(x).split("|") if g.strip()!=""])

print("===> Làm sạch hoàn tất. Số phim còn lại:", len(movies))


============ LÀM SẠCH DỮ LIỆU ============
===> Làm sạch hoàn tất. Số phim còn lại: 37944


In [10]:
sec("ĐỒNG BỘ HÓA RATING - MOVIES")

before = len(ratings)
ratings = ratings[ratings["MovieID"].isin(movies["MovieID"])]
print(f"Đã loại {before - len(ratings)} ratings trỏ tới MovieID không tồn tại")

ratings = ratings.dropna(subset=["Datetime"])
ratings = ratings[ratings["Rating"].between(0,10)]
print("Ratings hợp lệ:", len(ratings))

data = ratings.merge(
    movies[["MovieID","Title","Genres","GenresList","YearFromTitle"]],
    on="MovieID", how="left"
)
print("Kích thước sau merge:", data.shape)


============ ĐỒNG BỘ HÓA RATING - MOVIES ============
Đã loại 2379 ratings trỏ tới MovieID không tồn tại
Ratings hợp lệ: 919019
Kích thước sau merge: (919019, 9)


before = len(ratings) lưu lại số dòng ban đầu của bảng ratings.

ratings["MovieID"].isin(movies["MovieID"]):
Kiểm tra xem MovieID trong bảng ratings có tồn tại trong bảng movies không.

Chỉ giữ lại những dòng hợp lệ (MovieID có trong danh sách phim).

Nếu ratings chứa đánh giá cho phim không có thông tin trong bảng movies (gọi là orphan ratings), ta loại bỏ chúng.

📘 Ví dụ:
Nếu có MovieID = 999999 trong ratings mà không có trong movies, thì dòng đó bị loại.

dropna(subset=["Datetime"])
→ Bỏ các dòng bị thiếu giá trị thời gian (Datetime).
Vì ta cần timestamp cho chia temporal split, nên không thể giữ dòng nào không có thời gian.

ratings["Rating"].between(0,10)
→ Lọc chỉ giữ các giá trị trong khoảng [0,10].
(Tránh trường hợp dữ liệu bị lỗi, ví dụ điểm -1 hoặc 15.)

Mục tiêu: Gộp thông tin từ bảng movies (mô tả phim) vào bảng ratings (hành vi người dùng).

on="MovieID": nối hai bảng dựa trên khóa chung là MovieID.

how="left": giữ tất cả các dòng của ratings,
nếu phim nào có thông tin thì thêm cột Title, Genres, GenresList, YearFromTitle;
nếu không có (rất hiếm) thì điền NaN.

Bảng data là tập dữ liệu tổng hợp hoàn chỉnh — chứa cả thông tin hành vi (UserID, Rating, Datetime) và thông tin nội dung (Title, Genres, Year).
Đây chính là đầu vào cuối cùng cho các mô hình gợi ý, đặc biệt là mô hình Hybrid Recommender System.

In [ ]:
sec("TEMPORAL CROSS-VALIDATION (PER-USER)")

data = data.sort_values(["UserID","Datetime"]) #Sắp xếp dữ liệu theo người dùng và thời gian
folds = {} #Khởi tạo biến chứa kết quả

for fold, ratio in enumerate(TRAIN_RATIOS, start=1): #Duyệt qua danh sách TRAIN_RATIOS = [0.7, 0.75, 0.8]
    train_parts, test_parts = [], []
    for uid, grp in data.groupby("UserID", sort=False): 
#groupby("UserID") gom tất cả các rating của từng user vào nhóm (grp).
#Duyệt qua từng user để chia riêng lẻ — đây chính là phần “per-user” trong Temporal CV.
        n = len(grp)
        if n < MIN_USER_INTERACTIONS:
            train_parts.append(grp)
            continue
#Đếm số lượng rating của user (n).
#Nếu ít hơn ngưỡng tối thiểu MIN_USER_INTERACTIONS = 5 → không tách test, vì test sẽ rỗng hoặc quá ít → giữ toàn bộ vào train.
#⚙️Lý do: tránh test rỗng cho user ít hoạt động.
        split_idx = int(np.floor(ratio * n))
        split_idx = min(max(1, split_idx), n-1)
        train_parts.append(grp.iloc[:split_idx])
        test_parts.append(grp.iloc[split_idx:])
    train = pd.concat(train_parts, ignore_index=True)
    test  = pd.concat(test_parts, ignore_index=True)
    folds[f"Fold_{fold}"] = {"train":train, "test":test}
    print(f"Fold {fold}: Train={len(train)} | Test={len(test)} | Ratio={ratio}")


============ TEMPORAL CROSS-VALIDATION (PER-USER) ============
Fold 1: Train=655319 | Test=263700 | Ratio=0.7
Fold 2: Train=698450 | Test=220569 | Ratio=0.75
Fold 3: Train=741040 | Test=177979 | Ratio=0.8


split_idx: vị trí cắt giữa train và test (theo thứ tự thời gian).
Ví dụ: nếu user có 10 rating, ratio=0.8 → split_idx = 8.
→ Train: 8 dòng đầu, Test: 2 dòng cuối.
min(max(1, split_idx), n-1) bảo đảm:
Có ít nhất 1 dòng train, ít nhất 1 dòng test (không bị trống).

Bộ dữ liệu vẫn là một tập duy nhất (921 398 rating).

Ba fold chỉ chia khác nhau theo thời gian – tức là mô phỏng việc hệ thống được “cập nhật” qua các mốc thời gian khác nhau.

Trong Temporal Cross-Validation per-user, việc tăng tỉ lệ huấn luyện (từ 70%, 75% đến 80%) không phải là thêm dữ liệu mới, mà là thay đổi ranh giới thời gian giữa quá khứ và tương lai.

Điều này giúp mô phỏng kịch bản hệ thống gợi ý được huấn luyện lại sau một thời gian hoạt động, khi có thêm các tương tác lịch sử của người dùng.

Như vậy, mặc dù tập dữ liệu gốc không thay đổi, mỗi fold vẫn đại diện cho một giai đoạn phát triển khác nhau của hệ thống theo trục thời gian.

→ Sau khi huấn luyện & đánh giá 3 fold, mày sẽ có 3 bộ kết quả (RMSE, Precision@K,…).
Từ đó:

- Tính trung bình và độ lệch chuẩn → độ ổn định của mô hình.

- Vẽ biểu đồ hiệu năng theo tỉ lệ train → xu hướng học theo thời gian.

Fold 1 (70/30):
- Train = 655 319
- Test = 263 700

=> Tổng cộng ≈ 919 019 (do một vài user <5 rating được giữ toàn bộ trong train, nên tổng hơi lệch chút)

Train chiếm ~70% dữ liệu.

In [13]:
sec("TỔNG KẾT CUỐI")

summary = {
    "n_users": data["UserID"].nunique(),
    "n_movies": data["MovieID"].nunique(),
    "n_ratings": len(data),
    "n_folds": len(folds),
}
for k,v in summary.items(): print(f"{k}: {v}")

for i,(name,fold) in enumerate(folds.items(),start=1):
    print(f"Fold {i}: train={len(fold['train'])}, test={len(fold['test'])}, "
          f"unique_users={fold['train']['UserID'].nunique()}")


============ TỔNG KẾT CUỐI ============
n_users: 71648
n_movies: 37944
n_ratings: 919019
n_folds: 3
Fold 1: train=655319, test=263700, unique_users=71648
Fold 2: train=698450, test=220569, unique_users=71648
Fold 3: train=741040, test=177979, unique_users=71648


Ý nghĩa học thuật:

- Mỗi fold có cùng số lượng người dùng (71 707), nghĩa là tất cả user đều có dữ liệu huấn luyện — rất tốt, không bị cold-start.

- Train tăng dần, test giảm dần đúng logic 70/30 → 80/20.

- Tổng số rating giữa train + test ≈ tổng gốc (đảm bảo không mất mát dữ liệu).

Tại sao temporal per-user split giúp tránh cold-start?
User nào có <5 rating → được giữ toàn bộ trong train, không có test.

User nào đủ rating → chia theo thời gian, nên cả train và test đều thuộc cùng một user.

Kết quả:

- Không có user mới trong test → tránh được user cold-start.

- Mỗi user đã có ít nhất vài dòng rating trong train → mô hình đã “biết” họ.

- Tương tự, mỗi movie trong test đều tồn tại trong train → tránh item cold-start

In [ ]:
#Kiểm tra cold-start
# Kiểm tra user hoặc movie trong test mà không có trong train
for i, fold in folds.items():
    cold_users = np.setdiff1d(fold['test']['UserID'].unique(),
                              fold['train']['UserID'].unique())
    cold_movies = np.setdiff1d(fold['test']['MovieID'].unique(),
                               fold['train']['MovieID'].unique())
    print(f"Fold {i}: {len(cold_users)} user mới, {len(cold_movies)} phim mới")

Fold Fold_1: 0 user mới, 5466 phim mới
Fold Fold_2: 0 user mới, 4481 phim mới
Fold Fold_3: 0 user mới, 3488 phim mới


In [15]:
# =========================================
# 🔒 LƯU CHECKPOINT DỮ LIỆU SAU XỬ LÝ
# =========================================
import time, json
from pathlib import Path

# 🕒 1. Tạo thư mục version (theo ngày giờ để tránh ghi đè)
version = time.strftime("%Y%m%d-%H%M%S")
base_dir = Path(f"data/processed/{version}")
base_dir.mkdir(parents=True, exist_ok=True)

# 💾 2. Lưu bảng dữ liệu đã clean & merge cuối cùng
data.to_parquet(base_dir / "data_clean.parquet", index=False)

# 💾 3. Lưu từng fold (train/test)
for i, (name, fold) in enumerate(folds.items(), start=1):
    fold["train"].to_parquet(base_dir / f"fold{i}_train.parquet", index=False)
    fold["test"].to_parquet(base_dir / f"fold{i}_test.parquet", index=False)

# 🧠 4. Lưu metadata (thông tin cấu hình & kích thước)
meta = {
    "version": version,
    "n_users": int(data["UserID"].nunique()),
    "n_movies": int(data["MovieID"].nunique()),
    "n_ratings": int(len(data)),
    "time_min": str(data["Datetime"].min()),
    "time_max": str(data["Datetime"].max()),
    "train_ratios": TRAIN_RATIOS,
    "min_user_interactions": MIN_USER_INTERACTIONS,
}
with open(base_dir / "meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu checkpoint thành công tại: {base_dir.resolve()}")

✅ Đã lưu checkpoint thành công tại: C:\Users\ADMIN\Documents\Khóa luận\MovieTweetings Dataset\data\processed\20251102-161934
